### LLaMA로 라벨 정제하기

In [28]:
import json
import pandas as pd

with open('key_maps.json', 'r', encoding='utf-8') as f:
    key_maps = json.load(f)

train = pd.read_csv('train-no_special-no_hanja-polluted_lv.csv').drop(columns=['Unnamed: 0'])
print(key_maps)
train[train['polluted_lv']<0.2].sample(5)

{'0': '문화', '1': '스포츠', '2': '정치', '3': '사회', '4': '기술', '5': '경제', '6': '국제'}


,ID,text,target,polluted_lv,no_hanja
1365,ynat-v1_train_01365,"與.41-|스트트랙 $상b가속…""국| (한 좌석 남아I다",2,0.173913,여41스트트랙 상b가속국 한 좌석 남아I다
412,ynat-v1_train_00412,바이오분야 원천기술개발에 올해 3천700억원 투자,5,0.148148,바이오분야 원천기술개발에 올해 3천700억원 투자
1826,ynat-v1_train_01826,KT 배터리 절감 기술로 갤럭시 S9 최대 55% 더 쓴다,4,0.193548,KT 배터리 절감 기술로 갤럭시 S9 최대 55 더 쓴다
2225,ynat-v1_train_02225,프로농구 1라운드 최우수선수에 현대모비스 라건아,0,0.038462,프로농구 1라운드 최우수선수에 현대모비스 라건아
125,ynat-v1_train_00125,2자배구}대k팀*코7진 구성 완료…트R}너에 정기혁,1,0.166667,2자배구대k팀코7진 구성 완료트R너에 정기혁


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B' # 'beomi/Llama-3-Open-Ko-8B'

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map='auto'
)

model.eval()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): L

In [31]:
keys = list(key_maps.items())
keys

[('0', '문화'),
 ('1', '스포츠'),
 ('2', '정치'),
 ('3', '사회'),
 ('4', '기술'),
 ('5', '경제'),
 ('6', '국제')]

In [43]:
from tqdm.notebook import tqdm

keys = list(key_maps.values())
keys_str = []
for idx, key in list(key_maps.items()):
    keys_str.append(f'{idx}: {key}')
keys_str = ', '.join(keys_str)

PROMPT = f'''기사 제목을 보고 주어진 분야 중 올바른 분야의 번호를 선택하세요.
- 기사 분야: {keys_str}
- 위 분야 중 올바른 분야의 번호를 선택하세요.
- 문맥적으로 올바르지 않은 데이터는 '불가'라고 출력하세요.
- 반드시 주어진 키워드 중 한 개만 선택하세요.'''

fewshot = [
    {"role": "user", "content": "구미 영천 등 경북 9개 시군 폭염주의보"},
    {"role": "assistant", "content" : "0"},
    {"role": "user", "content": "울산 진보3당 북구시설관리공단 설립 조례안 철회하라"},
    {"role": "assistant", "content": "2"},
    {"role": "user", "content": "국제유가곡물가올해 농업경영비 33 감소 전망"},
    {"role": "assistant", "content": "3"}
]

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# train['new_target'] = None
for data in tqdm(train.iterrows(), desc='labeling', total=len(train)):
    data = data[1]
    
    if data['polluted_lv'] < 0.2:
        messages = [{"role": "system", "content": PROMPT}] + fewshot + [{"role": "user", "content": data['no_hanja']}]
        
        input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors='pt'
        ).to(model.device)

        outputs = model.generate(
            input_ids,
            max_new_tokens=1024,
            eos_token_id=terminators,
            pad_token_id=tokenizer.eos_token_id,
            do_sample=True,
            temperature=0.6,
            top_p=0.9
        )

        # train.loc[train['ID']==data['ID'], 'new_target'] = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        result = tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True)
        if result != '불가':
            train.loc[train['ID']==data['ID'], 'target'] = int(result)
train.head()

labeling:   0%|          | 0/2800 [00:00<?, ?it/s]

,ID,text,target,polluted_lv,no_hanja
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4,0.344828,정i 파1 미사z KT 이용기간 2e 단 Q분종U2보
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3,0.500000,K찰국DLwo 로L3한N 회장 2 T0송
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2,0.210526,m 김정 자주통일 새r열1나가야1보
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,1,0.107143,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩
4,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6,0.269231,pI미대선I앞두고 R2fr단 발 비해 감시 강화


In [44]:
train.to_csv('train-relabeled.csv')